# TP - Théorie des Tests Statistiques


Nous allons explorer :
1. Tests de comparaison de moyennes

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 1. Tests de comparaison de moyennes

Les tests de comparaison de moyennes permettent de déterminer s'il existe une différence significative entre les moyennes de deux populations.

### 1.1 Théorie

Nous avons deux approches principales pour comparer les moyennes de deux populations :

#### Cas 1 : Lorsque les variances σ²ₓ et σ²ᵧ sont connues

La fonction pivotale (statistique de test) est :

$U_0 = \frac{\bar{X}_{n_X} - \bar{Y}_{n_Y}}{\sqrt{\frac{\sigma^2_X}{n_X} + \frac{\sigma^2_Y}{n_Y}}} \sim \mathcal{N}(0, 1)$

Les différentes hypothèses et régions critiques sont :

| (H₀) | (H₁) | Région Critique |
|------|------|----------------|
| μₓ = μᵧ | μₓ ≠ μᵧ | RCα = {\|U₀\| > z_{α/2}} |
| μₓ = μᵧ ou μₓ ≥ μᵧ | μₓ < μᵧ | RCα = {U₀ < -z_{α}} |
| μₓ = μᵧ ou μₓ ≤ μᵧ | μₓ > μᵧ | RCα = {U₀ > z_{α}} |

où z_β est le quantile d'ordre 1-β de la loi normale centrée réduite.

#### Cas 2 : Lorsque les variances σ²ₓ et σ²ᵧ sont inconnues

La fonction pivotale (statistique de test) devient :

$T_0 = \frac{\bar{X}_{n_X} - \bar{Y}_{n_Y}}{\nu \sqrt{\frac{1}{n_X} + \frac{1}{n_Y}}} \sim T_m$

Avec $\nu^2 = \frac{(n_X-1)S^2_X + (n_Y-1)S^2_Y}{n_X+n_Y-2}$ et m = nₓ + nᵧ - 2 degrés de liberté.

Les différentes hypothèses et régions critiques sont :

| (H₀) | (H₁) | Région Critique |
|------|------|----------------|
| μₓ = μᵧ | μₓ ≠ μᵧ | RCα = {\|T₀\| > t_{m,α/2}} |
| μₓ = μᵧ ou μₓ ≥ μᵧ | μₓ < μᵧ | RCα = {T₀ < -t_{m,α}} |
| μₓ = μᵧ ou μₓ ≤ μᵧ | μₓ > μᵧ | RCα = {T₀ > t_{m,α}} |

où t_{m,β} est le quantile d'ordre 1-β de la loi de Student à m degrés de liberté.

### 1.2 Implémentation des tests de comparaison de moyennes

Nous allons maintenant implémenter ces tests en Python et les appliquer sur des exemples.

In [ ]:
def test_moyennes_variances_connues(X, Y, sigma2_X, sigma2_Y, alpha=0.05, alternative='two-sided'):
    """
    Test de comparaison de moyennes lorsque les variances sont connues
    
    Paramètres :
    X : tableau numpy, échantillon de la population X
    Y : tableau numpy, échantillon de la population Y
    sigma2_X : float, variance connue de la population X
    sigma2_Y : float, variance connue de la population Y
    alpha : float, niveau de signification (défaut: 0.05)
    alternative : string, type d'hypothèse alternative ('two-sided', 'less', 'greater')
    
    Retourne :
    U0 : float, valeur de la statistique de test
    p_value : float, p-valeur du test
    critical_value : float, valeur critique
    reject_H0 : bool, décision de rejeter H0 ou non
    """
    # Calcul des tailles d'échantillon
    n_X = len(X)
    n_Y = len(Y)
    
    # Calcul des moyennes d'échantillon
    X_bar = np.mean(X)
    Y_bar = np.mean(Y)
    
    # Calcul de la statistique de test U0
    denominateur = np.sqrt(sigma2_X/n_X + sigma2_Y/n_Y)
    U0 = (X_bar - Y_bar) / denominateur
    
    # Calcul de la valeur critique et de la p-valeur en fonction du type d'alternative
    if alternative == 'two-sided':
        critical_value = stats.norm.ppf(1 - alpha/2)
        p_value = 2 * (1 - stats.norm.cdf(abs(U0)))
        reject_H0 = abs(U0) > critical_value
    elif alternative == 'less':
        critical_value = stats.norm.ppf(alpha)
        p_value = stats.norm.cdf(U0)
        reject_H0 = U0 < critical_value
    elif alternative == 'greater':
        critical_value = stats.norm.ppf(1 - alpha)
        p_value = 1 - stats.norm.cdf(U0)
        reject_H0 = U0 > critical_value
    else:
        raise ValueError("'alternative' doit être 'two-sided', 'less' ou 'greater'")
    
    return U0, p_value, critical_value, reject_H0

In [ ]:
def test_moyennes_variances_inconnues(X, Y, alpha=0.05, alternative='two-sided', equal_var=True):
    """
    Test de comparaison de moyennes lorsque les variances sont inconnues
    
    Paramètres :
    X : tableau numpy, échantillon de la population X
    Y : tableau numpy, échantillon de la population Y
    alpha : float, niveau de signification (défaut: 0.05)
    alternative : string, type d'hypothèse alternative ('two-sided', 'less', 'greater')
    equal_var : bool, si True, suppose que les variances sont égales (défaut: True)
    
    Retourne :
    T0 : float, valeur de la statistique de test
    p_value : float, p-valeur du test
    critical_value : float, valeur critique
    reject_H0 : bool, décision de rejeter H0 ou non
    nu : float, estimation de l'écart-type poolé
    """
    # Calcul des tailles d'échantillon
    n_X = len(X)
    n_Y = len(Y)
    
    # Calcul des moyennes d'échantillon
    X_bar = np.mean(X)
    Y_bar = np.mean(Y)
    
    # Calcul des variances d'échantillon
    S2_X = np.var(X, ddof=1)  # ddof=1 pour utiliser la formule corrigée (n-1)
    S2_Y = np.var(Y, ddof=1)
    
    # Calcul des degrés de liberté
    df = n_X + n_Y - 2
    
    if equal_var:
        # Calcul de nu^2 (variance poolée) sous l'hypothèse de variances égales
        nu2 = ((n_X - 1) * S2_X + (n_Y - 1) * S2_Y) / df
        nu = np.sqrt(nu2)
        
        # Calcul de T0
        T0 = (X_bar - Y_bar) / (nu * np.sqrt(1/n_X + 1/n_Y))
    else:
        # Test de Welch pour variances inégales
        var_X = S2_X / n_X
        var_Y = S2_Y / n_Y
        T0 = (X_bar - Y_bar) / np.sqrt(var_X + var_Y)
        
        # Calcul des degrés de liberté de Welch
        df = ((var_X + var_Y)**2) / ((var_X**2 / (n_X - 1)) + (var_Y**2 / (n_Y - 1)))
        nu = None  # Non applicable dans ce cas
    
    # Calcul de la valeur critique et de la p-valeur en fonction du type d'alternative
    if alternative == 'two-sided':
        critical_value = stats.t.ppf(1 - alpha/2, df)
        p_value = 2 * (1 - stats.t.cdf(abs(T0), df))
        reject_H0 = abs(T0) > critical_value
    elif alternative == 'less':
        critical_value = stats.t.ppf(alpha, df)
        p_value = stats.t.cdf(T0, df)
        reject_H0 = T0 < critical_value
    elif alternative == 'greater':
        critical_value = stats.t.ppf(1 - alpha, df)
        p_value = 1 - stats.t.cdf(T0, df)
        reject_H0 = T0 > critical_value
    else:
        raise ValueError("'alternative' doit être 'two-sided', 'less' ou 'greater'")
    
    return T0, p_value, critical_value, reject_H0, nu

### 1.3 Exemples d'application des tests de comparaison de moyennes

Maintenant, appliquons ces tests sur des données simulées.

In [ ]:
# Générateur de nombres aléatoires pour la reproductibilité
np.random.seed(0)

# Exemple 1: Deux populations avec des moyennes différentes et variances connues
mu_X = 10
mu_Y = 12
sigma_X = 3
sigma_Y = 2.5
n_X = 30
n_Y = 25

# Génération des échantillons
X = np.random.normal(mu_X, sigma_X, n_X)
Y = np.random.normal(mu_Y, sigma_Y, n_Y)

# Affichage des statistiques descriptives
print("Échantillon X:")
print(f"  Moyenne: {np.mean(X):.4f}")
print(f"  Écart-type: {np.std(X, ddof=1):.4f}")
print(f"  Taille: {len(X)}")
print("\nÉchantillon Y:")
print(f"  Moyenne: {np.mean(Y):.4f}")
print(f"  Écart-type: {np.std(Y, ddof=1):.4f}")
print(f"  Taille: {len(Y)}")

# Visualisation des distributions
plt.figure(figsize=(12, 6))
sns.histplot(X, kde=True, color='blue', alpha=0.5, label='Échantillon X')
sns.histplot(Y, kde=True, color='red', alpha=0.5, label='Échantillon Y')
plt.axvline(np.mean(X), color='blue', linestyle='--', label=f'Moyenne X: {np.mean(X):.2f}')
plt.axvline(np.mean(Y), color='red', linestyle='--', label=f'Moyenne Y: {np.mean(Y):.2f}')
plt.title('Distribution des échantillons X et Y')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Test de comparaison des moyennes avec variances connues
# Utilisons les vraies variances des populations
sigma2_X = sigma_X**2
sigma2_Y = sigma_Y**2

# Test bilatéral: H0: μₓ = μᵧ contre H1: μₓ ≠ μᵧ
U0, p_value, critical_value, reject_H0 = test_moyennes_variances_connues(
    X, Y, sigma2_X, sigma2_Y, alpha=0.05, alternative='two-sided')

print("Test de comparaison des moyennes avec variances connues (test bilatéral)")
print(f"H0: μₓ = μᵧ contre H1: μₓ ≠ μᵧ")
print(f"Statistique U0: {U0:.4f}")
print(f"Valeur critique à α=0.05: ±{critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'Il existe une différence significative entre les moyennes' if reject_H0 else 'Il n\'y a pas de différence significative entre les moyennes'}")

# Visualisation de la distribution de la statistique de test sous H0
x = np.linspace(-4, 4, 1000)
y = stats.norm.pdf(x, 0, 1)

plt.figure(figsize=(12, 6))
plt.plot(x, y, 'b-', label='N(0,1)')
plt.fill_between(x, y, where=(x >= critical_value) | (x <= -critical_value), color='red', alpha=0.3, label=f'Région critique (α=0.05)')
plt.axvline(U0, color='red', linestyle='--', label=f'U0 = {U0:.4f}')
plt.title('Distribution de la statistique de test U0 sous H0')
plt.xlabel('U0')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Test unilatéral à gauche: H0: μₓ = μᵧ ou μₓ ≥ μᵧ contre H1: μₓ < μᵧ
U0, p_value, critical_value, reject_H0 = test_moyennes_variances_connues(
    X, Y, sigma2_X, sigma2_Y, alpha=0.05, alternative='less')

print("Test de comparaison des moyennes avec variances connues (test unilatéral à gauche)")
print(f"H0: μₓ = μᵧ ou μₓ ≥ μᵧ contre H1: μₓ < μᵧ")
print(f"Statistique U0: {U0:.4f}")
print(f"Valeur critique à α=0.05: {critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'La moyenne de X est significativement inférieure à celle de Y' if reject_H0 else 'On ne peut pas conclure que la moyenne de X est inférieure à celle de Y'}")

# Test unilatéral à droite: H0: μₓ = μᵧ ou μₓ ≤ μᵧ contre H1: μₓ > μᵧ
U0, p_value, critical_value, reject_H0 = test_moyennes_variances_connues(
    X, Y, sigma2_X, sigma2_Y, alpha=0.05, alternative='greater')

print("\nTest de comparaison des moyennes avec variances connues (test unilatéral à droite)")
print(f"H0: μₓ = μᵧ ou μₓ ≤ μᵧ contre H1: μₓ > μᵧ")
print(f"Statistique U0: {U0:.4f}")
print(f"Valeur critique à α=0.05: {critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'La moyenne de X est significativement supérieure à celle de Y' if reject_H0 else 'On ne peut pas conclure que la moyenne de X est supérieure à celle de Y'}")

### 1.4 Test avec variances inconnues

Maintenant, supposons que nous ne connaissons pas les variances des populations et utilisons notre deuxième fonction.

In [ ]:
# Exemple 2: Deux populations avec des moyennes différentes et variances inconnues
# On utilise les mêmes données que précédemment mais on prétend ne pas connaître les variances

# Test bilatéral: H0: μₓ = μᵧ contre H1: μₓ ≠ μᵧ
T0, p_value, critical_value, reject_H0, nu = test_moyennes_variances_inconnues(
    X, Y, alpha=0.05, alternative='two-sided', equal_var=True)

# Degrés de liberté
df = len(X) + len(Y) - 2

print("Test de Student pour la comparaison des moyennes avec variances inconnues (test bilatéral)")
print(f"H0: μₓ = μᵧ contre H1: μₓ ≠ μᵧ")
print(f"Statistique T0: {T0:.4f}")
print(f"Degrés de liberté: {df}")
print(f"Écart-type poolé (ν): {nu:.4f}")
print(f"Valeur critique à α=0.05: ±{critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'Il existe une différence significative entre les moyennes' if reject_H0 else 'Il n\'y a pas de différence significative entre les moyennes'}")

# Visualisation de la distribution de la statistique de test sous H0
x = np.linspace(-4, 4, 1000)
y = stats.t.pdf(x, df)

plt.figure(figsize=(12, 6))
plt.plot(x, y, 'b-', label=f'Student(df={df})')
plt.fill_between(x, y, where=(x >= critical_value) | (x <= -critical_value), color='red', alpha=0.3, label=f'Région critique (α=0.05)')
plt.axvline(T0, color='red', linestyle='--', label=f'T0 = {T0:.4f}')
plt.title('Distribution de la statistique de test T0 sous H0')
plt.xlabel('T0')
plt.ylabel('Densité de probabilité')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Test unilatéral à gauche: H0: μₓ = μᵧ ou μₓ ≥ μᵧ contre H1: μₓ < μᵧ
T0, p_value, critical_value, reject_H0, nu = test_moyennes_variances_inconnues(
    X, Y, alpha=0.05, alternative='less', equal_var=True)

print("Test de Student pour la comparaison des moyennes avec variances inconnues (test unilatéral à gauche)")
print(f"H0: μₓ = μᵧ ou μₓ ≥ μᵧ contre H1: μₓ < μᵧ")
print(f"Statistique T0: {T0:.4f}")
print(f"Valeur critique à α=0.05: {critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'La moyenne de X est significativement inférieure à celle de Y' if reject_H0 else 'On ne peut pas conclure que la moyenne de X est inférieure à celle de Y'}")

# Test unilatéral à droite: H0: μₓ = μᵧ ou μₓ ≤ μᵧ contre H1: μₓ > μᵧ
T0, p_value, critical_value, reject_H0, nu = test_moyennes_variances_inconnues(
    X, Y, alpha=0.05, alternative='greater', equal_var=True)

print("\nTest de Student pour la comparaison des moyennes avec variances inconnues (test unilatéral à droite)")
print(f"H0: μₓ = μᵧ ou μₓ ≤ μᵧ contre H1: μₓ > μᵧ")
print(f"Statistique T0: {T0:.4f}")
print(f"Valeur critique à α=0.05: {critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'La moyenne de X est significativement supérieure à celle de Y' if reject_H0 else 'On ne peut pas conclure que la moyenne de X est supérieure à celle de Y'}")

# Test unilatéral à droite: H0: μₓ = μᵧ ou μₓ ≤ μᵧ contre H1: μₓ > μᵧ
T0, p_value, critical_value, reject_H0, nu = test_moyennes_variances_inconnues(
    X, Y, alpha=0.05, alternative='greater', equal_var=False)

print("\nTest de Student pour la comparaison des moyennes avec variances inconnues (test unilatéral à droite)")
print(f"H0: μₓ = μᵧ ou μₓ ≤ μᵧ contre H1: μₓ > μᵧ")
print(f"Statistique T0: {T0:.4f}")
print(f"Valeur critique à α=0.05: {critical_value:.4f}")
print(f"p-valeur: {p_value:.4f}")
print(f"Décision: {'Rejeter H0' if reject_H0 else 'Ne pas rejeter H0'}")
print(f"Conclusion: {'La moyenne de X est significativement supérieure à celle de Y' if reject_H0 else 'On ne peut pas conclure que la moyenne de X est supérieure à celle de Y'}")